In [20]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
# File to Load (Remember to Change These)
file_to_load = "output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load, encoding="utf-8")

# Store csv created in part one into a DataFrame

df.insert(0, 'City_ID', range(0, len(df)))
df['City'] = df['City'].str.lower()
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,84.20,74,0,8.05,BR,1616705605
1,1,jamestown,42.0970,-79.2353,68.00,37,1,8.08,US,1616705891
2,2,rikitea,-23.1203,-134.9692,79.54,73,61,18.41,PF,1616705845
3,3,brooks,50.5834,-111.8851,32.00,69,100,12.66,CA,1616705893
4,4,lavrentiya,65.5833,-171.0000,3.00,84,28,2.46,RU,1616705894


In [22]:
len(df)

578

In [23]:
df.dropna().count()

City_ID       574
City          574
Lat           574
Lng           574
Max Temp      574
Humidity      574
Cloudiness    574
Wind Speed    574
Country       574
Date          574
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humiditymap = df["Humidity"].astype(float)

In [26]:
# Heatmap of humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humiditymap, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Narrow down cities that fit criteria and drop any results with null values

df = df.dropna()

In [34]:
# df['Max Temp'] = maxtemp

# for temp in maxtemp:
#     if maxtemp < 70 | maxtemp > 80
    

# df['Max Temp'] = df['Max Temp'] > 70 and df['Max Temp'] < 80
# df

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates


,City,Country,Lat,Lng,Hotel Name
81,sakit,IN,27.45,78.82,
103,ormara,PK,25.21,64.64,
215,tura,IN,25.52,90.22,
274,chabahar,IR,25.29,60.64,
308,myitkyina,MM,25.38,97.40,
422,ghotki,PK,28.01,69.32,
484,nalut,LY,30.33,10.85,
544,sabha,LY,27.04,14.43,


In [7]:
# Set parameters to search for a hotel


# Iterate through 

    # get lat, lng from df

    
    # Use the search term: "Hotel" and our lat/lng
   

    # make request and print url
    
    
    # convert to json
   
    
    # Grab the first hotel from the results and store the name




Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
81,sakit,IN,27.45,78.82,Laucky house
103,ormara,PK,25.21,64.64,
215,tura,IN,25.52,90.22,Hotel RIKMAN Continental
274,chabahar,IR,25.29,60.64,هتل گدروشیا
308,myitkyina,MM,25.38,97.40,Shwe Phyu Hotel
422,ghotki,PK,28.01,69.32,Mohalla Old Bazar
484,nalut,LY,30.33,10.85,
544,sabha,LY,27.04,14.43,محطة ام الارانب -سبها مواصلات


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))